In [ ]:
import pandas as pd

In [2]:


import pandas as pd
import mysql.connector
import os

csv_files = [
    ('pizza_sales.csv', 'Pizza_Sales')
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='pizza_sales'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'C:/Users/USER/jupyternotebook/pizza_sales'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    df = pd.read_csv(file_path)
    
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    for _, row in df.iterrows():
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    conn.commit()

conn.close()

Processing pizza_sales.csv
NaN values before replacement:
pizza_id             0
order_id             0
pizza_name_id        0
quantity             0
order_date           0
order_time           0
unit_price           0
total_price          0
pizza_size           0
pizza_category       0
pizza_ingredients    0
pizza_name           0
dtype: int64



'hello'

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
db=mysql.connector.connect(host="localhost",
                           username="root",
                           password="",
                           database="pizza_sales")
cur=db.cursor()


In [5]:
query="""
select sum(total_price) as Total_revenue
from pizza_sales
"""
cur.execute(query)
data=cur.fetchall()
print("The total Revenue is :",data[0][0])


The total Revenue is : 817860.05


In [6]:
query="""
SELECT COUNT(DISTINCT order_id) AS total_orders
FROM pizza_sales;

"""
cur.execute(query)
data=cur.fetchall()
print("The total Pizzas Order :",data[0][0])


The total Pizzas Order : 21350


In [9]:
query=""" select sum(quantity) as total_pazzas_Sold
from pizza_sales"""
cur.execute(query)
data=cur.fetchall()
print("The total pizzas Sold :",data[0][0]);

The total pizzas Sold : 49574


In [10]:
query=""" select sum(total_price)/count(distinct order_id) as Avg_order_value
from pizza_sales"""
cur.execute(query)
data=cur.fetchall()
print("The Average Order Values :",data[0][0]);

The Average Order Values : 38.307262


In [29]:
query = """
SELECT pizza_name, SUM(quantity) AS total_sold
FROM pizza_sales
GROUP BY pizza_name
ORDER BY total_sold DESC
LIMIT 5
"""

cur.execute(query)

# Get column names
columns = [desc[0] for desc in cur.description]

# Get data
data = cur.fetchall()

# Print table
print(f"{columns[0]:30} | {columns[1]}")
print("-" * 45)

for row in data:
    print(f"{row[0]:30} | {row[1]}")


pizza_name                     | total_sold
---------------------------------------------
The Classic Deluxe Pizza       | 2453
The Barbecue Chicken Pizza     | 2432
The Hawaiian Pizza             | 2422
The Pepperoni Pizza            | 2418
The Thai Chicken Pizza         | 2371


In [30]:
pip install prettytable


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [33]:


query = """
SELECT pizza_name, SUM(quantity) AS total_sold
FROM pizza_sales
GROUP BY pizza_name
ORDER BY total_sold DESC
LIMIT 5
"""
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='pizza_sales'
)

df = pd.read_sql(query, conn)
df


C:\Users\USER\AppData\Local\Temp\ipykernel_16348\2113791090.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,pizza_name,total_sold
0,The Classic Deluxe Pizza,2453.0
1,The Barbecue Chicken Pizza,2432.0
2,The Hawaiian Pizza,2422.0
3,The Pepperoni Pizza,2418.0
4,The Thai Chicken Pizza,2371.0


In [36]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine("mysql+pymysql://root:@localhost/pizza_sales")

query = """
SELECT pizza_name, SUM(quantity) AS total_sold
FROM pizza_sales
GROUP BY pizza_name
ORDER BY total_sold DESC
LIMIT 5
"""

df = pd.read_sql(query, engine)
print("Top 5 Best Selling Pizzas");
df


Top 5 Best Selling Pizzas


,pizza_name,total_sold
0,The Classic Deluxe Pizza,2453.0
1,The Barbecue Chicken Pizza,2432.0
2,The Hawaiian Pizza,2422.0
3,The Pepperoni Pizza,2418.0
4,The Thai Chicken Pizza,2371.0


In [40]:
query = """
SELECT pizza_category,sum(total_price) As Revenue
FROM pizza_sales
GROUP BY pizza_Category
ORDER BY Revenue Desc
"""
df=pd.read_sql(query,engine)
print("Category wise total revenue")
df

Category wise total revenue


,pizza_category,Revenue
0,Classic,220053.10
1,Supreme,208197.00
2,Chicken,195919.50
3,Veggie,193690.45


In [46]:
query = """
SELECT pizza_Size, SUM(total_price) AS Total_Revenue 
FROM pizza_sales
GROUP BY pizza_size
ORDER BY Total_revenue DESC
"""
df=pd.read_sql(query,engine)
df

,pizza_Size,Total_Revenue
0,L,375318.70
1,M,249382.25
2,S,178076.50
3,XL,14076.00
4,XXL,1006.60


In [55]:
query = """
SELECT Order_date, SUM(total_price) AS Daily_revenue_$
FROM pizza_sales
GROUP BY Order_date
ORDER BY order_date
limit 50
"""
df=pd.read_sql(query,engine)
df

,Order_date,Daily_revenue_$
0,01-01-2015,2713.85
1,01-02-2015,3189.20
2,01-03-2015,1598.55
3,01-04-2015,2176.85
4,01-05-2015,2571.95
5,01-06-2015,3067.75
6,01-07-2015,2231.50
7,01-08-2015,2440.55
8,01-09-2015,2352.85
9,01-10-2015,3202.15


In [61]:
query = """
SELECT 
    HOUR(order_time) AS order_hour,
    COUNT(DISTINCT order_id) AS total_orders
FROM pizza_sales
GROUP BY hour(order_time)
ORDER BY order_hour asc;


"""

df=pd.read_sql(query,engine)
df


,order_hour,total_orders
0,9,1
1,10,8
2,11,1231
3,12,2520
4,13,2455
5,14,1472
6,15,1468
7,16,1920
8,17,2336
9,18,2399


In [62]:
query="""
select pizza_name,sum(total_price) as Revenue
from pizza_sales
group by pizza_name
order by revenue desc
limit 1;"""
df=pd.read_sql(query,engine)
df

,pizza_name,Revenue
0,The Thai Chicken Pizza,43434.25
